In [1]:
#import
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import time


In [ ]:

#URL
url_nakaku = 'http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22131&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'
url_higashiku = 'http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22132&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'
url_nishiku = 'http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22133&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'
url_hamakitaku = 'http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22136&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'
url_kitaku = 'http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22135&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'
url_minamiku = 'http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22134&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'
url_tenryuku = 'http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22137&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'

#url = url_tenryuku


In [16]:
def makecsv(csvfilename, url, wait):
    #get http contents
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c, 'html.parser')
    summary = soup.find("div",{'id':'js-bukkenList'})
    body = soup.find("body")
    pages = body.find_all("div",{'class':'pagination pagination_set-nav'})
    pages_text = str(pages)
    pages_split = pages_text.split('</a></li>\n</ol>')
    pages_split0 = pages_split[0]
    pages_split1 = pages_split0[-3:]
    pages_split2 = pages_split1.replace('>','')
    try:
        pages_split3 = int(pages_split2)
    except:
        pages_split3 = 2 #Tenryu-ku
    #URLを入れるリスト
    urls = []

    #1ページ目を格納
    urls.append(url)

    #2ページ目から最後のページまでを格納
    for i in range(pages_split3-1):
        pg = str(i+2)
        url_page = url + '&pn=' + pg
        urls.append(url_page)

    name = [] #マンション名
    address = [] #住所
    locations0 = [] #立地1つ目（最寄駅/徒歩~分）
    locations1 = [] #立地2つ目（最寄駅/徒歩~分）
    locations2 = [] #立地3つ目（最寄駅/徒歩~分）
    age = [] #築年数
    height = [] #建物高さ
    floor = [] #階
    rent = [] #賃料
    admin = [] #管理費
    others = [] #敷/礼/保証/敷引,償却
    floor_plan = [] #間取り
    area = [] #専有面積

    #print(pages_split3)
    #print(urls)
    
    #各ページで以下の動作をループ
    for url in urls:
        print("read url ... ", url[-3:].strip("ipn="), " / ", pages_split3)
        #物件リストを切り出し
        result = requests.get(url)
        c = result.content
        soup = BeautifulSoup(c, 'html.parser')
        summary = soup.find("div",{'id':'js-bukkenList'})

        #マンション名、住所、立地（最寄駅/徒歩~分）、築年数、建物高さが入っているcassetteitemを全て抜き出し
        cassetteitems = summary.find_all("div",{'class':'cassetteitem'})

        #各cassetteitemsに対し、以下の動作をループ
        for i in range(len(cassetteitems)):
            #各建物から売りに出ている部屋数を取得
            tbodies = cassetteitems[i].find_all('tbody')

            #マンション名取得
            subtitle = cassetteitems[i].find_all("div",{
                'class':'cassetteitem_content-title'})
            subtitle = str(subtitle)
            subtitle_rep = subtitle.replace(
                '[<div class="cassetteitem_content-title">', '')
            subtitle_rep2 = subtitle_rep.replace(
                '</div>]', '')

            #住所取得
            subaddress = cassetteitems[i].find_all("li",{
                'class':'cassetteitem_detail-col1'})
            subaddress = str(subaddress)
            subaddress_rep = subaddress.replace(
                '[<li class="cassetteitem_detail-col1">', '')
            subaddress_rep2 = subaddress_rep.replace(
                '</li>]', '')

            #部屋数だけ、マンション名と住所を繰り返しリストに格納（部屋情報と数を合致させるため）
            for y in range(len(tbodies)):
                name.append(subtitle_rep2)
                address.append(subaddress_rep2)

            #立地を取得
            sublocations = cassetteitems[i].find_all("li",{
                'class':'cassetteitem_detail-col2'})

            #立地は、1つ目から3つ目までを取得（4つ目以降は無視）
            for x in sublocations:
                cols = x.find_all('div')
                for i in range(len(cols)):
                    text = cols[i].find(text=True)
                    for y in range(len(tbodies)):
                        if i == 0:
                            locations0.append(text)
                        elif i == 1:
                            locations1.append(text)
                        elif i == 2:
                            locations2.append(text)

            #築年数と建物高さを取得
            tbodies = cassetteitems[i].find_all('tbody')
            col3 = cassetteitems[i].find_all("li",{
                'class':'cassetteitem_detail-col3'})
            for x in col3:
                cols = x.find_all('div')
                for i in range(len(cols)):
                    text = cols[i].find(text=True)
                    for y in range(len(tbodies)):
                        if i == 0:
                            age.append(text)
                        else:
                            height.append(text)

        #階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積が入っているtableを全て抜き出し
        tables = summary.find_all('table')

        #各建物（table）に対して、売りに出ている部屋（row）を取得
        rows = []
        for i in range(len(tables)):
            rows.append(tables[i].find_all('tr'))

        #各部屋に対して、tableに入っているtext情報を取得し、dataリストに格納
        data = []
        for row in rows:
            for tr in row:
                cols = tr.find_all('td')
                for td in cols:
                    text = td.find(text=True)
                    data.append(text)

        #dataリストから、階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積を順番に取り出す
        index = 0
        for item in data:
            if '階' in item:
                floor.append(data[index])
                rent.append(data[index+1])
                admin.append(data[index+2])
                others.append(data[index+3])
                floor_plan.append(data[index+4])
                area.append(data[index+5])
            index +=1

        #プログラムを10秒間停止する（スクレイピングマナー）
        time.sleep(wait)

    #各リストをシリーズ化
    name = Series(name)
    address = Series(address)
    locations0 = Series(locations0)
    locations1 = Series(locations1)
    locations2 = Series(locations2)
    age = Series(age)
    height = Series(height)
    floor = Series(floor)
    rent = Series(rent)
    admin = Series(admin)
    others = Series(others)
    floor_plan = Series(floor_plan)
    area = Series(area)

    #各シリーズをデータフレーム化
    suumo_df = pd.concat([name, address, locations0, locations1,
                          locations2, age, height,floor,rent,admin,others,floor_plan,area],axis=1)

    #カラム名
    suumo_df.columns=['マンション名','住所','立地1','立地2','立地3','築年数','建物高さ','階','賃料','管理費',
                      '敷/礼/保証/敷引,償却','間取り','専有面積']

    #csvファイルとして保存
    suumo_df.to_csv(csvfilename, sep = '\t',encoding='utf-16')

In [3]:
makecsv("tenryuku.csv", url_tenryuku, 5)

In [5]:
makecsv("hamakitaku.csv", url_hamakitaku, 5)

In [7]:
makecsv("minamiku.csv", url_minamiku, 5)

In [9]:
makecsv("nishiku.csv", url_nishiku, 5)

Start getting data from URL =  http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22133&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1
Start getting data from URL =  http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22133&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&pn=2
Start getting data from URL =  http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22133&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&pn=3
Start getting data from URL =  http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22133&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&pn=4
Start getting data from URL =  http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=050&bs=040&ta=22&sc=22133&cb=0.0&ct=9999

In [11]:
makecsv("kitaku.csv", url_kitaku, 5)

max page count =  28
Now start at  i=  page.
max page count =  28
Now start at  n=  page.
max page count =  28
Now start at  n=  page.
max page count =  28
Now start at  n=  page.
max page count =  28
Now start at  n=  page.
max page count =  28
Now start at  n=  page.
max page count =  28
Now start at  n=  page.
max page count =  28
Now start at  n=  page.
max page count =  28
Now start at  n=  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =1  page.
max page count =  28
Now start at  =2  page.
max page count =  28
Now start at  =2  page.
max page count =  28
Now start at  =2  page.
max page c

In [13]:
makecsv("nakaku.csv", url_nakaku, 5)

max page count =  183
Now start at  =1  page.
max page count =  183
Now start at  =2  page.
max page count =  183
Now start at  =3  page.
max page count =  183
Now start at  =4  page.
max page count =  183
Now start at  =5  page.
max page count =  183
Now start at  =6  page.
max page count =  183
Now start at  =7  page.
max page count =  183
Now start at  =8  page.
max page count =  183
Now start at  =9  page.
max page count =  183
Now start at  10  page.
max page count =  183
Now start at  11  page.
max page count =  183
Now start at  12  page.
max page count =  183
Now start at  13  page.
max page count =  183
Now start at  14  page.
max page count =  183
Now start at  15  page.
max page count =  183
Now start at  16  page.
max page count =  183
Now start at  17  page.
max page count =  183
Now start at  18  page.
max page count =  183
Now start at  19  page.
max page count =  183
Now start at  20  page.
max page count =  183
Now start at  21  page.
max page count =  183
Now start at

max page count =  183
Now start at  80  page.
max page count =  183
Now start at  81  page.
max page count =  183
Now start at  82  page.
max page count =  183
Now start at  83  page.


In [17]:
makecsv("higashiku.csv", url_higashiku, 5)

read url ...  1  /  76
read url ...  2  /  76
read url ...  3  /  76
read url ...  4  /  76
read url ...  5  /  76
read url ...  6  /  76
read url ...  7  /  76
read url ...  8  /  76
read url ...  9  /  76
read url ...  10  /  76
read url ...  11  /  76
read url ...  12  /  76
read url ...  13  /  76
read url ...  14  /  76
read url ...  15  /  76
read url ...  16  /  76
read url ...  17  /  76
read url ...  18  /  76
read url ...  19  /  76
read url ...  20  /  76
read url ...  21  /  76
read url ...  22  /  76
read url ...  23  /  76
read url ...  24  /  76
read url ...  25  /  76
read url ...  26  /  76
read url ...  27  /  76
read url ...  28  /  76
read url ...  29  /  76
read url ...  30  /  76
read url ...  31  /  76
read url ...  32  /  76
read url ...  33  /  76
read url ...  34  /  76
read url ...  35  /  76
read url ...  36  /  76
read url ...  37  /  76
read url ...  38  /  76
read url ...  39  /  76
read url ...  40  /  76
read url ...  41  /  76
read url ...  42  /  76
r